In [28]:
import pandas as pd
import numpy as np

In [29]:
pandemic = pd.read_csv("final/Pandemic_v1.csv")
prepandemic = pd.read_csv("final/prepandemic_v1.csv")
unemployment = pd.read_csv("final/Unemployment_v1.csv")

In [30]:
pandemic.describe()

,Unnamed: 0,stfips,ctyfips,fips,relegiousGathering,AlcoholStores_open,firearms_open,Medical_expansion,Paid_sick_leaves,confirmed_cases_20200122,...,deaths_20200409,deaths_20200410,deaths_20200411,deaths_20200412,deaths_20200413,deaths_20200414,deaths_20200415,deaths_20200416,deaths_20200417,deaths_20200418
count,3220.000000,3220.000000,3220.000000,3220.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3220.000000,...,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000
mean,1664.174534,31.290683,102.922050,31393.605280,0.355824,0.984087,0.830045,0.577339,0.106620,0.000311,...,5.007143,5.604658,6.242236,6.727019,7.184161,7.874845,8.677019,10.102174,11.110870,11.541615
std,958.795196,16.277207,106.693846,16292.078954,0.478839,0.125161,0.375653,0.494061,0.308679,0.017623,...,93.778425,105.999094,116.054463,125.679428,133.989094,144.332531,154.642829,207.019970,237.322764,237.748522
min,1.000000,1.000000,1.000000,1001.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,835.750000,19.000000,35.000000,19032.500000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1662.500000,30.000000,79.000000,30024.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2497.250000,46.000000,133.000000,46105.500000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,3322.000000,72.000000,840.000000,72153.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,5150.000000,5820.000000,6367.000000,6898.000000,7349.000000,7905.000000,8455.000000,11477.000000,13202.000000,13202.000000


In [31]:
prepandemic.describe()

,stfips,ctyfips,fips,uninsured_2013,uninsured_2016,uninsured_2017,medexp_2017,agiprop1_2017,agiprop2_2017,agiprop3_2017,...,ageg16_2018,ageg17_2018,ageg18_2018,male_2018,Hispanic2018,nH_White_2018,nH_Black_2018,nH_Indian_Na_2018,nH_Asian_2018,nH_Hawaii_Na_2018
count,3142.000000,3142.000000,3142.000000,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,...,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000
mean,30.280076,103.572884,30383.649268,0.143053,0.088750,0.090884,0.476918,0.014535,0.086591,0.139517,...,0.035036,0.023463,0.023855,0.501146,0.096529,0.759925,0.089932,0.019444,0.014945,0.001053
std,15.144339,107.704060,15162.508374,0.038468,0.034182,0.035369,0.499546,0.011968,0.017175,0.025240,...,0.009838,0.007096,0.009051,0.022832,0.138368,0.201938,0.142989,0.074052,0.028851,0.009541
min,1.000000,1.000000,1001.000000,0.037000,0.025000,0.028000,0.000000,0.000000,0.000000,0.000000,...,0.005591,0.000000,0.000000,0.431295,0.006105,0.026913,0.000000,0.000000,0.000000,0.000000
25%,18.000000,35.000000,18177.500000,0.113000,0.061000,0.061000,0.000000,0.008067,0.077238,0.125015,...,0.028830,0.018889,0.018032,0.489747,0.024005,0.643114,0.007087,0.002427,0.004249,0.000183
50%,29.000000,79.000000,29176.000000,0.140000,0.087000,0.085000,0.000000,0.011187,0.084773,0.139673,...,0.034200,0.022811,0.022580,0.496895,0.044005,0.833735,0.022273,0.003853,0.006749,0.000363
75%,45.000000,133.000000,45080.500000,0.171000,0.104000,0.107000,1.000000,0.016798,0.093201,0.153704,...,0.040004,0.027160,0.027835,0.505897,0.099994,0.922958,0.102109,0.007781,0.013406,0.000684
max,56.000000,840.000000,56045.000000,0.221000,0.166000,0.173000,1.000000,0.136426,0.275000,0.306407,...,0.133821,0.068182,0.159091,0.731649,0.963596,0.978872,0.854143,0.905095,0.427881,0.488636


In [32]:
unemployment

,stname,stfips,ctyname,ctyfips,fips,laborforce_2019_1,unemployment_2019_1,laborforce_2019_2,unemployment_2019_2,laborforce_2019_3,...,neighbor4_2017,neighbor5_2017,neighbor6_2017,neighbor7_2017,neighbor8_2017,neighbor9_2017,neighbor10_2017,neighbor11_2017,neighbor12_2017,neighbor13_2017
0,Alabama,1,Autauga County,1,1001,25808.0,3.9,26084.0,3.5,26269.0,...,1101.0,1021.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,1,Baldwin County,3,1003,92160.0,4.1,93498.0,3.7,95982.0,...,12033.0,1025.0,1129.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,1,Barbour County,5,1005,8250.0,5.3,8339.0,4.9,8378.0,...,13259.0,1045.0,13239.0,1113.0,1067.0,NaN,NaN,NaN,NaN,NaN
3,Alabama,1,Bibb County,7,1007,8705.0,4.4,8770.0,4.1,8825.0,...,1117.0,1073.0,1021.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alabama,1,Blount County,9,1009,25024.0,3.9,25262.0,3.5,25471.0,...,1073.0,1127.0,1055.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Alabama,1,Bullock County,11,1011,4604.0,5.5,4732.0,4.3,4766.0,...,1113.0,1101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Alabama,1,Butler County,13,1013,8835.0,5.3,8988.0,4.6,9128.0,...,1041.0,1035.0,1039.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Alabama,1,Calhoun County,15,1015,45893.0,5.4,46078.0,4.6,46263.0,...,1029.0,1055.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Alabama,1,Chambers County,17,1017,15233.0,4.3,15373.0,3.8,15360.0,...,1123.0,1111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Alabama,1,Cherokee County,19,1019,11486.0,4.2,11646.0,4.3,11594.0,...,1029.0,1015.0,13055.0,1055.0,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
confirmed_cases = pandemic['confirmed_cases_20200418']
deaths = pandemic['deaths_20200418']
state = pandemic['stname']
county = pandemic['ctyname']

In [34]:
populationdensity=prepandemic['popdensity_2018']

In [46]:
populationdensity_min = populationdensity.min()
populationdensity_mean = populationdensity.mean()
populationdensity_stage = populationdensity_mean - populationdensity_min
def populationdensity_category(populationdensity):
    if populationdensity < populationdensity_mean - populationdensity_stage * 0.9: 
        return '1_Very Low' 
    elif populationdensity < populationdensity_mean - populationdensity_stage * 0.3: 
        return '2_Low'  
    elif populationdensity < populationdensity_mean + populationdensity_stage * 0.3: 
        return '3_Medium'   
    elif populationdensity < populationdensity_mean + populationdensity_stage: 
        return '4_High' 
    else:
        return 'Very_High'
populationdensity_category = prepandemic.apply(lambda x: populationdensity_category(x['popdensity_2018']), axis = 1)

In [72]:
def etnic_group_top1(hispanic, white, black, indian, asian, hawaii_na):
    ethnic_groups = [('Hispanic', hispanic), ('White', white), ('Black', black), ('Indian', indian), ('Asian', asian), ('Hawaii and N/A', hawaii_na)]
    ethnic_groups = sorted(ethnic_groups, key = lambda x: -x[1])
    return ethnic_groups[0][0]
def etnic_group_top2(hispanic, white, black, indian, asian, hawaii_na):
    ethnic_groups = [('Hispanic', hispanic), ('White', white), ('Black', black), ('Indian', indian), ('Asian', asian), ('Hawaii and N/A', hawaii_na)]
    ethnic_groups = sorted(ethnic_groups, key = lambda x: -x[1])
    return ethnic_groups[1][0]
def etnic_group_top3(hispanic, white, black, indian, asian, hawaii_na):
    ethnic_groups = [('Hispanic', hispanic), ('White', white), ('Black', black), ('Indian', indian), ('Asian', asian), ('Hawaii and N/A', hawaii_na)]
    ethnic_groups = sorted(ethnic_groups, key = lambda x: -x[1])
    return ethnic_groups[2][0]
ethnic_group_top1 = prepandemic.apply(lambda x: etnic_group_top1(x['Hispanic2018'], x['nH_White_2018'], x['nH_Black_2018'], x['nH_Indian_Na_2018'], x['nH_Asian_2018'], x['nH_Hawaii_Na_2018']), axis = 1)
ethnic_group_top2 = prepandemic.apply(lambda x: etnic_group_top2(x['Hispanic2018'], x['nH_White_2018'], x['nH_Black_2018'], x['nH_Indian_Na_2018'], x['nH_Asian_2018'], x['nH_Hawaii_Na_2018']), axis = 1)
ethnic_group_top3 = prepandemic.apply(lambda x: etnic_group_top3(x['Hispanic2018'], x['nH_White_2018'], x['nH_Black_2018'], x['nH_Indian_Na_2018'], x['nH_Asian_2018'], x['nH_Hawaii_Na_2018']), axis = 1)

0       Hispanic
1       Hispanic
2       Hispanic
3       Hispanic
4          Black
5       Hispanic
6       Hispanic
7       Hispanic
8       Hispanic
9       Hispanic
10      Hispanic
11      Hispanic
12      Hispanic
13      Hispanic
14      Hispanic
15      Hispanic
16      Hispanic
17      Hispanic
18      Hispanic
19      Hispanic
20      Hispanic
21         Black
22      Hispanic
23      Hispanic
24         Black
25      Hispanic
26        Indian
27      Hispanic
28      Hispanic
29         Black
          ...   
3112      Indian
3113       Asian
3114       Asian
3115      Indian
3116       Black
3117       Asian
3118       Asian
3119       Asian
3120      Indian
3121      Indian
3122       Black
3123      Indian
3124       Black
3125    Hispanic
3126       Black
3127      Indian
3128      Indian
3129       Black
3130      Indian
3131       Black
3132      Indian
3133       Asian
3134       Asian
3135      Indian
3136      Indian
3137       Black
3138       Asian
3139       Bla